# Data Masters: Case

In [1]:
FIT_MODELS = True

## Bibliotecas

In [2]:
# --- Data Exploration and Viz --- #
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Classification models --- #
from sklearn.ensemble import \
    RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb

# --- Pipeline Building --- #
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from resources.fit_refit import FitValidateRefit as FVR

# --- Model Evaluation --- #
#from resources.custommetrics import profit
from sklearn.metrics import \
    roc_auc_score, \
    recall_score, \
    precision_score, \
    f1_score, \
    confusion_matrix

# --- Tuning --- #
from sklearn.model_selection import \
    GridSearchCV, \
    train_test_split

# --- Preprocessing --- #
from sklearn.preprocessing import \
    StandardScaler, \
    OrdinalEncoder, \
    FunctionTransformer
from sklearn.impute import SimpleImputer
from resources.customtransformers import \
    DropConstantColumns, \
    DropDuplicateColumns, \
    AddNoneCount, \
    AddNonZeroCount, \
    CustomImputer

# --- Cluster Analysis --- #
#from sklearn.decomposition import PCA
#from sklearn.cluster import KMeans

## Leitura dos dados

In [3]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

## Definição dos pipelines de pré-processamento

### Pipeline 1: Mantendo dados originais

Este pipeline serve como um "one size fits all" para os passos seguintes. Como as missing features do conjunto de dados já foram preenchidas artificalmente, tratá-las para modelos de árvore não necessariamente trará melhora na performance, uma vez que o preenchimento é padronizado por prefixo e estes modelos lidam bem com relações não-lineares entre variáveis. Assim, a partir deste pipeline, podemos iniciar os testes de diferentes modelos baseados em árvores.

In [4]:
prep_01 = Pipeline(
    steps=[
        ("dcc", DropConstantColumns()),
        ("ddc", DropDuplicateColumns()),
        (
            "anc_delta",
            AddNoneCount(
                prefix="delta",
                fake_value=9999999999
            )
        ),
        ("anzc_saldo", AddNonZeroCount(prefix="saldo")),
        ("anzc_imp", AddNonZeroCount(prefix="imp")),
        ("anzc_delta", AddNonZeroCount(prefix="delta")),
        ("anzc_ind", AddNonZeroCount(prefix="ind")),
        (
            "col_specific",
            ColumnTransformer(
                [
                    (
                        "ord_encoders",
                        OrdinalEncoder(
                            handle_unknown="use_encoded_value",
                            unknown_value=-1,
                            encoded_missing_value=-1,
                            min_frequency=40
                        ),
                        ["var36","var21"]                    
                    )
                ],
                remainder="passthrough"
            )
        )
    ]
)

### Pipeline 2: Imputando nulos com mediana

No caso das variáveis delta, temos dados extremamente esparsos. Assim, optou-se por imputar os valores faltantes com a mediana, uma vez que a média seria aftetada por outliers. Essencialmente, estamos preenchendo estes valores com 0 e colocando-os no mesmo "grupo" que a maioria das observações.

In [5]:
prep_02 = Pipeline(
    steps=[
        ("dcc", DropConstantColumns()),
        ("ddc", DropDuplicateColumns()),
        (
            "impute_none_delta",
            CustomImputer(
                prefix="delta",
                value=9999999999
            )
        ),
        ("anc_delta", AddNoneCount(prefix="delta")),
        (
            "impute_nan_var3",
            CustomImputer(
                prefix="var3",
                value=-999999
            )
        ),
        ("anzc_saldo", AddNonZeroCount(prefix="saldo")),
        ("anzc_imp", AddNonZeroCount(prefix="imp")),
        ("anzc_delta", AddNonZeroCount(prefix="delta")),
        ("anzc_ind", AddNonZeroCount(prefix="ind")),
        (
            "col_specific",
            ColumnTransformer(
                [
                    (
                        "ord_encoders",
                        OrdinalEncoder(
                            handle_unknown="use_encoded_value",
                            unknown_value=-1,
                            encoded_missing_value=-1,
                            min_frequency=40
                        ),
                        ["var36","var21"]                
                    )
                ],
                remainder="passthrough"
            )
        )
    ]
)

### Pipeline 3: Permitindo valores nulos

O XGBoost permite que sejam incluídos valores nulos. Em cada split -- caso haja algum valor nulo -- o algoritmo decidirá com base em sua função de custo para qual direção do split os valores nulos devem ser enviados.

## Definição da métrica de avaliação

Os modelos (com excessão de uma árvore simples inicial) passaram por validação cruzada para hiperparametrização buscando maximizar a AUC -- métrica que, de modo geral, indica quão bem o modelo consegue separar as classes da variável "TARGET" ao comparar as estimativas com os valores reais com diferentes cortes de classificação. O corte de classificação, por sua vez, foi ajustado sobre o modelo campeão (com a maior AUC) com base na métrica sugerida pelo enunciado deste trabalho:

$Profit = 90tp -10fp$

In [6]:
def profit(y_true, y_pred):
    tp = np.sum((y_pred == 1) & (y_true == 1))
    fp = np.sum((y_pred == 1) & (y_true == 0))
    n = len(y_true)
    return (-10 * fp + 90 * tp) / n

## Avaliação dos modelos

In [7]:
scores = {}

### Modelo base: árvore de classificação

Aqui definimos um ponto de partida para os modelos de classificação. Foi escolhida a árvore de classificação pois esta servirá como base dos dois outros algorítmos testados (Random Forest e Gradient Boosting). Este modelo não passa por gridsearch e seu resultado é considerado o mínimo a ser batido pelos outros modelos. 

In [8]:
dtc = Pipeline(
    steps=[
        ("preprocessing", prep_02),
        ("clf", DecisionTreeClassifier())
    ]
)

dtc

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('dcc', DropConstantColumns()),
                                 ('ddc', DropDuplicateColumns()),
                                 ('impute_none_delta',
                                  CustomImputer(prefix='delta',
                                                value=9999999999)),
                                 ('anc_delta', AddNoneCount(prefix='delta')),
                                 ('impute_nan_var3',
                                  CustomImputer(prefix='var3', value=-999999)),
                                 ('anzc_saldo',
                                  AddNonZeroCount(prefix='saldo')),
                                 ('anzc_imp', AddNon...)),
                                 ('anzc_delta',
                                  AddNonZeroCount(prefix='delta')),
                                 ('anzc_ind', AddNonZeroCount(prefix='ind')),
                                 ('col_specific',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('ord_encoders',
                                                                   OrdinalEncoder(encoded_missing_value=-1,
                                                                                  handle_unknown='use_encoded_value',
                                                                                  min_frequency=40,
                                                                                  unknown_value=-1),
                                                                   ['var36',
                                                                    'var21'])]))])),
                ('clf', DecisionTreeClassifier())])

In [9]:
simple_tree_model = FVR(
    train.drop("ID", axis=1),
    dtc,
    "TARGET",
    test_size=.25,
    use_grid_search=False
)

In [10]:
if FIT_MODELS == True:
    simple_tree_model.fit()
    simple_tree_model.save_model("models/simple_tree_model.pkl")

In [11]:
simple_tree_model = simple_tree_model

y_pred = simple_tree_model \
    .predict(
        test.drop(["ID","TARGET"],
        axis=1)
    )

scores["dtc"] = profit(y_pred, test["TARGET"])

### Modelo desafiante 1: Floresta Aleatória

In [ ]:
scores

{'dtc': 0.20257826887661143, 'rfc': 0.1696921862667719}